In [58]:
# Imports 
import numpy as np
import pandas as pd
import pyodbc as db
import matplotlib.pyplot as plt
import os
%matplotlib inline
import requests
from requests.auth import HTTPBasicAuth
from io import StringIO
from sodapy import Socrata
from datetime import date
import datetime
import seaborn as sns
import configparser
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Constants 
stringVAERSDATA     = "VAERSDATA"
stringVAERSSYMPTOMS = "VAERSSYMPTOMS"
stringVAERSVAX      = "VAERSVAX"
inputDirectory      = '.\RAW_DATA'
outputDirectory     = '.\OUTPUT'
pklInputDirectory   = '.\INPUT'

usePickle           = 0

In [2]:
# Locals 
dfVAERSDATA_Initialized     = False
dfVAERSSYMPTOMS_Initialized = False
dfVAERSVAX_Initialized      = False

# @TODO Column datatypes - need to finish  - getting the following warning when reading in files:
# DtypeWarning: Columns (11,13,15,16,27) have mixed types.Specify dtype option on import or set low_memory=False. interactivity=interactivity, compiler=compiler, result=result)
dtype_VAERSDATA = {"VAERS_ID": np.int64, "RECVDATE": np.object, "STATE": np.str, "AGE_YRS": np.float64, "CAGE_YR": np.float64, "CAGE_MO": np.float64, "SEX": np.str, "RPT_VDATE": np.object, "SYMPTOM_TEXT": np.str, "DIED": np.str, "DATEDIED": np.object, "L_THREAD": np.str, "ER_VISIT": np.str}

# Iterate over files 
for filename in os.listdir(inputDirectory):

    # Create the file name 
    fullFilename = os.path.join(inputDirectory, filename)

    # Ensure its a file
    if os.path.isfile(fullFilename):

        # Print the file name to see where it breaks - if needed 
        # print(fullFilename)

        # Overall
        if(fullFilename.__contains__(stringVAERSDATA)):
            
            # Check if the dataframe has been initialized
            if(dfVAERSDATA_Initialized == False):
                
                # Read in the file to this dataframe directly 
                dfVAERSDATA = pd.read_csv(fullFilename, encoding='cp1252', dtype=dtype_VAERSDATA)

                # Set to init 
                dfVAERSDATA_Initialized = True

            else:

                # Add the new file to the dataframe that has been init'd
                dfVAERSDATA = pd.concat([dfVAERSDATA, pd.read_csv(fullFilename, encoding='cp1252', dtype=dtype_VAERSDATA)], ignore_index=True)
            
        # Symptoms
        elif(fullFilename.__contains__(stringVAERSSYMPTOMS)):
            
            # Check if the dataframe has been initialized
            if(dfVAERSSYMPTOMS_Initialized == False):
                
                # Read in the file to this dataframe directly 
                dfVAERSSYMPTOMS = pd.read_csv(fullFilename, encoding='cp1252')

                # Set to init 
                dfVAERSSYMPTOMS_Initialized = True

            else:

                # Add the new file to the dataframe that has been init'd
                dfVAERSSYMPTOMS = pd.concat([dfVAERSSYMPTOMS, pd.read_csv(fullFilename, encoding='cp1252')], ignore_index=True)

        # Vax
        elif(fullFilename.__contains__(stringVAERSVAX)):
            
            # Check if the dataframe has been initialized
            if(dfVAERSVAX_Initialized == False):
                
                # Read in the file to this dataframe directly 
                dfVAERSVAX = pd.read_csv(fullFilename, encoding='cp1252')

                # Set to init 
                dfVAERSVAX_Initialized = True

            else:

                # Add the new file to the dataframe that has been init'd
                dfVAERSVAX = pd.concat([dfVAERSVAX, pd.read_csv(fullFilename, encoding='cp1252')], ignore_index=True)
        
completeDataFrame = pd.merge(left = dfVAERSVAX, right = dfVAERSDATA, \
                      how = "inner", left_on = ["VAERS_ID"], \
                      right_on = ["VAERS_ID"])

# Create a pickle 
completeDataFrame.to_pickle(os.path.join(outputDirectory, "completeDataFrame.pkl"))

<ipython-input-2-e47fde80ed8c>:8: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype_VAERSDATA = {"VAERS_ID": np.int64, "RECVDATE": np.object, "STATE": np.str, "AGE_YRS": np.float64, "CAGE_YR": np.float64, "CAGE_MO": np.float64, "SEX": np.str, "RPT_VDATE": np.object, "SYMPTOM_TEXT": np.str, "DIED": np.str, "DATEDIED": np.object, "L_THREAD": np.str, "ER_VISIT": np.str}
<ipython-input-2-e47fde80ed8c>:8: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release

In [40]:
# Use the pickle or not
if(usePickle == True):

    # Read in the pickle 
    rawVaxFrame = pd.read_pickle(os.path.join(pklInputDirectory, "completeDataFrame.pkl"))

else:

    # Just make a copy
    rawVaxFrame = completeDataFrame.copy()

# Show the columns 
# rawVaxFrame.info()

In [53]:
# Make a copy 
workingCovidFrame = rawVaxFrame.copy(deep=True)

# Only get the covid vaxes 
workingCovidFrame = workingCovidFrame[workingCovidFrame["VAX_TYPE"] == "COVID19"]

# Drop the duplicates 
workingCovidFrame = workingCovidFrame.drop_duplicates(subset=["VAERS_ID"])

# Get rid of all columns except those we want 
workingCovidFrame = workingCovidFrame[['VAX_MANU', 'VAX_DOSE_SERIES', 'STATE', 'AGE_YRS', 'CAGE_YR', 'SEX', 'NUMDAYS', 'DIED']]

# Print this out 
# workingCovidFrame.to_csv(os.path.join(outputDirectory, "workingCovidFrame.csv"))

In [56]:
# If NA then assign as 0 else they died
workingCovidFrame["DIED"] = np.where(workingCovidFrame["DIED"].isna(), 0, 1)

# If NA then assign as 0 else they died
workingCovidFrame["AGE_YRS"] = np.where(workingCovidFrame["AGE_YRS"].isna(), workingCovidFrame["CAGE_YR"], workingCovidFrame["AGE_YRS"])

# If NA then assign as 0 else they died
workingCovidFrame["CAGE_YR"] = np.where(workingCovidFrame["CAGE_YR"].isna(), workingCovidFrame["AGE_YRS"], workingCovidFrame["CAGE_YR"])

# Lose na for years old 
workingCovidFrame = workingCovidFrame[workingCovidFrame["AGE_YRS"].isna() == False]

# Drop the NA values from all other columns
workingCovidFrame = workingCovidFrame.dropna()

In [67]:
workingCovidFrame["NUMDAYS"].unique()

array([0.0000e+00, 1.0000e+00, 2.0000e+00, 3.0000e+00, 1.6000e+01,
       1.7000e+01, 1.8411e+04, 1.9000e+01, 4.0000e+00, 6.0000e+00,
       1.8628e+04, 5.0000e+00, 2.0000e+01, 7.0000e+00, 1.9822e+04,
       6.2000e+01, 2.2000e+01, 2.1000e+01, 2.3000e+01, 3.1000e+01,
       1.4000e+01, 1.5000e+01, 8.0000e+00, 9.0000e+00, 2.6000e+01,
       2.5000e+01, 1.0000e+01, 1.2000e+01, 2.7000e+01, 2.8000e+01,
       1.3000e+01, 1.1000e+01, 1.6072e+04, 2.9000e+01, 3.0000e+01,
       3.7000e+01, 1.5404e+04, 3.6000e+01, 3.7100e+02, 3.2000e+01,
       3.6600e+02, 3.6300e+02, 3.3000e+01, 3.6800e+02, 2.4000e+01,
       7.3120e+03, 1.8000e+01, 3.6100e+02, 3.4000e+01, 7.3140e+03,
       3.6900e+02, 2.4100e+02, 3.2890e+03, 3.3700e+02, 3.6700e+02,
       3.6500e+02, 2.5760e+04, 3.4200e+02, 3.6525e+04, 2.5059e+04,
       7.3090e+03, 3.2880e+03, 3.6529e+04, 2.1090e+04, 3.5000e+01,
       4.1000e+01, 3.7200e+02, 6.1000e+01, 2.4618e+04, 2.0323e+04,
       3.3800e+02, 3.7300e+02, 1.9766e+04, 3.7000e+02, 3.6519e

In [59]:
# Break apart the X and y 
xRaw = workingCovidFrame.drop(columns=["DIED"])
yRaw = workingCovidFrame[["DIED"]]

# Split using test train
X_train, X_test, y_train, y_test = train_test_split( xRaw, yRaw, test_size=0.33, random_state=42)

In [60]:
# Create the logistic model 
log_reg = LogisticRegression()

# Fit the data 
log_reg.fit(X_train, y_train)




ValueError: could not convert string to float: 'JANSSEN'